### Jira Tickets Solution
This notebook describes a solution for Jira Tickets creanted in `create_jira_tickets.py`

In [1]:
import logging
import os
import sys
from pathlib import Path

import pandas as pd

# since notebook is outside of the src directory, we need to add the src directory to the path
project_root = Path.home() / "dev" / "data-analyser"
if project_root not in sys.path:
    sys.path.append(str(project_root))

from src.clients.db_client import DatabaseClient
from src.agent.agent import DataAnalysisAgent
from scripts.constants import TICKETS_TO_TEST

logging.basicConfig(level=logging.INFO)

In [2]:
# db connection
DB_PATH = os.path.expanduser("../data/porsche_analytics.db")
sqlite_connection_string = f"sqlite:///{DB_PATH}"

sqlite_client = DatabaseClient(sqlite_connection_string)
print(f"Connected to the database!")

Connected to the database!


In [3]:
agent = DataAnalysisAgent(
    agent_config=str(project_root / "config" / "config.yaml"),
    db_client=sqlite_client,
    max_retries=3,
)

In [4]:
# get ground truth results for testing tickets
ground_truth = {}

for ticket in TICKETS_TO_TEST:
    print(f"Processing ticket: {ticket['id']} - {ticket['summary']}")
    query = ticket.get("sql")
    if not query:
        logging.warning(f"No SQL query provided for ticket {ticket['id']}. Skipping.")
        continue

    try:
        result = sqlite_client.execute_query(query)
        ground_truth[ticket['id']] = pd.DataFrame(result.data)

    except Exception as e:
        logging.error(f"Error processing ticket {ticket['id']}: {e}")

INFO:src.clients.db_client:Query executed successfully. Returned 7 rows.
INFO:src.clients.db_client:Query executed successfully. Returned 4 rows.
INFO:src.clients.db_client:Query executed successfully. Returned 1 rows.
INFO:src.clients.db_client:Query executed successfully. Returned 4 rows.
INFO:src.clients.db_client:Query executed successfully. Returned 10 rows.
INFO:src.clients.db_client:Query executed successfully. Returned 3 rows.
INFO:src.clients.db_client:Query executed successfully. Returned 10 rows.
INFO:src.clients.db_client:Query executed successfully. Returned 10 rows.


INFO:src.clients.db_client:Query executed successfully. Returned 8 rows.
INFO:src.clients.db_client:Query executed successfully. Returned 10 rows.
INFO:src.clients.db_client:Query executed successfully. Returned 6 rows.
INFO:src.clients.db_client:Query executed successfully. Returned 10 rows.
INFO:src.clients.db_client:Query executed successfully. Returned 10 rows.
INFO:src.clients.db_client:Query executed successfully. Returned 0 rows.
INFO:src.clients.db_client:Query executed successfully. Returned 10 rows.
INFO:src.clients.db_client:Query executed successfully. Returned 10 rows.


Processing ticket: 1 - Car Models Analysis
Processing ticket: 2 - Dealership Performance by Region Analysis
Processing ticket: 3 - Count Electric Models
Processing ticket: 4 - Top Regions by Dealerships
Processing ticket: 5 - Average Sale Price by Region and Model
Processing ticket: 6 - Sales Count by Payment Method
Processing ticket: 7 - Popular Models by Region
Processing ticket: 8 - Revenue by Dealership
Processing ticket: 9 - Average Age of Customers per Country
Processing ticket: 10 - Service Cost by Dealership
Processing ticket: 11 - Model Price vs Horsepower
Processing ticket: 12 - Top Performing Dealerships
Processing ticket: 13 - Profit Margin by Model
Processing ticket: 14 - Repeat Customers Analysis
Processing ticket: 15 - Service Frequency by Model
Processing ticket: 16 - Lifetime Value by Customer


In [5]:
ground_truth[1]

,segment,models_unq_count
0,Sports Car,5
1,SUV,3
2,Wagon,1
3,Supercar,1
4,Sedan,1
5,Luxury,1
6,Hypercar,1


In [6]:
# get results from Agent for testing tickets
from_agent = {}

for ticket in TICKETS_TO_TEST:
    print(f"Processing ticket: {ticket['id']} - {ticket['summary']}")
    try:
        sql_from_agent = agent.sql_generation_tool.generate_query(
            task_description=ticket['description']
        )

        # valdation logic
        validation_result = agent.sql_validation_tool.validate_sql(sql_query=sql_from_agent)
        if validation_result.is_valid:
            result = agent.db_client.execute_query(sql_query=sql_from_agent)
        else:
            logging.warning(f"SQL query for ticket {ticket['id']} is not valid: {validation_result.error_message}")
            continue

        from_agent[ticket['id']] = pd.DataFrame(result.data)
        from_agent['sql'] = sql_from_agent

    except Exception as e:
        logging.error(f"Error processing ticket {ticket['id']}: {e}")

INFO:src.tools.sql_tool:Generating SQL query for task: How many unqiue car models we have per car category? Sort the results in descending order by models_unq_count!


Processing ticket: 1 - Car Models Analysis


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:src.clients.db_client:Query executed successfully. Returned 7 rows.
INFO:src.tools.sql_tool:Generating SQL query for task: Analyze the average dealership rating and sales capacity by region. Which regions have the highest performing dealerships? Sort the results by average rating in descending order.


Processing ticket: 2 - Dealership Performance by Region Analysis


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:src.clients.db_client:Query executed successfully. Returned 4 rows.
INFO:src.tools.sql_tool:Generating SQL query for task: How many electric car models are available in the database? Sort the results in descending order by electric_model_count.


...

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:src.clients.db_client:Query executed successfully. Returned 0 rows.
INFO:src.tools.sql_tool:Generating SQL query for task: For each car model, how many service records exist per 10.000 km on average? Sort the results by 'service_records_per_10000_km' in descending order.


Processing ticket: 15 - Service Frequency by Model


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:src.clients.db_client:Query executed successfully. Returned 13 rows.
INFO:src.tools.sql_tool:Generating SQL query for task: Calculate the lifetime value of each customer based on total purchases and service costs. Sort the results by 'lifetime_value' in descending order. Include only top 10 customers.


Processing ticket: 16 - Lifetime Value by Customer


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
ERROR:src.clients.db_client:Error executing query: (sqlite3.OperationalError) no such column: sr.customer_id
[SQL: SELECT 
    c.customer_id, 
    c.first_name, 
    c.last_name, 
    SUM(s.price + sr.cost) AS lifetime_value
FROM 
    customers c
LEFT JOIN 
    sales s ON c.customer_id = s.customer_id
LEFT JOIN 
    service_records sr ON c.customer_id = sr.customer_id
GROUP BY 
    c.customer_id, 
    c.first_name, 
    c.last_name
ORDER BY 
    lifetime_value DESC
LIMIT 10;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
ERROR:root:Error processing ticket 16: (sqlite3.OperationalError) no such column: sr.customer_id
[SQL: SELECT 
    c.customer_id, 
    c.first_name, 
    c.last_name, 
    SUM(s.price + sr.cost) AS lifetime_value
FROM 
    customers c
LEFT JOIN 
    sales s ON c.customer_id = s.customer_id
LEFT JOIN 
    service_records sr ON c.customer_id = sr.customer_id
GROUP BY

In [7]:
result = {
    'ticket_id': [],
    'difficulty': [],
    'is_correct': [],
}

for ticket_id in TICKETS_TO_TEST:
    try:
        ground_truth_result = ground_truth[ticket_id['id']]
        from_agent_result = from_agent[ticket_id['id']]
        is_correct = (ground_truth_result.values == from_agent_result.values).all().item() * 1

        result['is_correct'].append(is_correct)
        result['ticket_id'].append(ticket_id['id'])
        result['difficulty'].append(ticket_id['difficulty'])
    except:
        continue

result_df = pd.DataFrame(result)
result_df

,ticket_id,difficulty,is_correct
0,1,easy,1
1,2,easy,1
2,3,easy,1
3,4,easy,1
4,5,easy,1
5,6,easy,1
6,7,medium,1
7,9,medium,0
8,11,medium,1
9,14,hard,1


In [29]:
i = 15
ground_truth_result = ground_truth[i]
from_agent_result = from_agent[i]

In [30]:
(ground_truth_result.values == from_agent_result.values).all().item() * 1

ValueError: operands could not be broadcast together with shapes (10,4) (13,3) 

In [31]:
ground_truth_result

,model_id,total_service_records,total_mileage,service_records_per_10000_km
0,4,1,5000,2.000000
1,1,1,5000,2.000000
2,6,1,6000,1.666667
3,2,1,7500,1.333333
4,3,1,8000,1.250000
5,11,1,9000,1.111111
6,10,1,10000,1.000000
7,9,1,10000,1.000000
8,7,1,12000,0.833333
9,5,1,15000,0.666667


In [32]:
from_agent_result

,model_id,model_name,service_records_per_10000_km
0,1,911 Carrera,2.000000
1,4,Panamera,2.000000
2,6,Macan,1.666667
3,2,911 Turbo S,1.333333
4,3,Taycan,1.250000
5,11,911 GT3,1.111111
6,9,Taycan Cross Turismo,1.000000
7,10,Cayenne Coupe,1.000000
8,7,718 Boxster,0.833333
9,5,Cayenne,0.666667


In [ ]:
i = 1
ground_truth_result = ground_truth[i]
from_agent_result = from_agent[i]

# # optiona A: ignore index/column order (but column names must match!)
# ground_truth_result.sort_index(axis=1).sort_index(axis=0).equals(
#     from_agent_result.sort_index(axis=1).sort_index(axis=0)
# )

# option B: check only data matching (looser validation)
(ground_truth_result.values == from_agent_result.values).all().item() * 1

1

### Task 1: Car Models Analysis
- How many unqiue car models we have per car category? Sort the results in descending order!

In [ ]:
query = """
SELECT
    segment,
    COUNT(DISTINCT model_code) AS models_unq_count
FROM models
GROUP BY segment
ORDER BY models_unq_count DESC
"""
result = sqlite_client.execute_query(query)
result_df = pd.DataFrame(result.data)
result_df

INFO:src.clients.db_client:Query executed successfully. Returned 7 rows.


,segment,models_unq_count
0,Sports Car,5
1,SUV,3
2,Wagon,1
3,Supercar,1
4,Sedan,1
5,Luxury,1
6,Hypercar,1


### Task 2: Dealership Performance by Region Analysis
- Analyze the average dealership rating and sales capacity by region. Which regions have the highest performing dealerships? Sort the results by average rating in descending order.

In [ ]:
query = """
SELECT
    region,
    ROUND(AVG(rating), 2) AS average_rating,
    ROUND(AVG(sales_capacity), 1) AS average_sales_capacity
FROM
    dealerships
GROUP BY
    region
ORDER BY
    average_rating DESC;
"""
result = sqlite_client.execute_query(query)
result_df = pd.DataFrame(result.data)
result_df

INFO:src.clients.db_client:Query executed successfully. Returned 4 rows.


,region,average_rating,average_sales_capacity,dealership_count
0,Middle East,4.80,40.0,1
1,North America,4.65,32.5,2
2,Europe,4.63,24.3,4
3,Asia Pacific,4.63,21.0,3


### Task 3: Count Electric Models
- How many electric car models are available in the database?

In [ ]:
query = """
SELECT COUNT(*) AS electric_model_count
FROM models
WHERE is_electric = 1;
"""
result = sqlite_client.execute_query(query)
result_df = pd.DataFrame(result.data)
result_df

INFO:src.clients.db_client:Query executed successfully. Returned 1 rows.


,electric_model_count
0,3


### Task 4: Top Regions by Dealerships
- Which regions have the most dealerships?

In [ ]:
query = """
SELECT region, COUNT(*) AS dealership_count
FROM dealerships
GROUP BY region
ORDER BY dealership_count DESC;
"""
result = sqlite_client.execute_query(query)
result_df = pd.DataFrame(result.data)
result_df

INFO:src.clients.db_client:Query executed successfully. Returned 4 rows.


,region,dealership_count
0,Europe,4
1,Asia Pacific,3
2,North America,2
3,Middle East,1


### Task 5: Average Sale Price by Region and Model
- For each region and car model, calculate the average sale price of all cars sold

In [ ]:
query = """
SELECT 
    d.region,
    m.model_name,
    AVG(s.price) AS average_sale_price
FROM sales s
JOIN dealerships d ON s.dealership_id = d.dealership_id
JOIN models m ON s.model_id = m.model_id
GROUP BY d.region, m.model_name
ORDER BY average_sale_price DESC;
"""
result = sqlite_client.execute_query(query)
result_df = pd.DataFrame(result.data)
result_df

INFO:src.clients.db_client:Query executed successfully. Returned 10 rows.


,region,model_name,average_sale_price
0,North America,911 Turbo S,215000.0
1,Middle East,Panamera,182000.0
2,Europe,911 GT3,167500.0
3,North America,911 Carrera,110500.0
4,Asia Pacific,Taycan Cross Turismo,98500.0
5,Europe,Taycan,92400.0
6,Europe,Cayenne Coupe,79900.0
7,Europe,Cayenne,72500.0
8,Asia Pacific,718 Boxster,65000.0
9,Asia Pacific,Macan,59800.0


### Task 6: Sales Count by Payment Method
- How many cars were sold using each payment method?

In [27]:
query = """
SELECT
    payment_method,
    COUNT(*) AS cars_sold
FROM sales
GROUP BY payment_method
ORDER BY cars_sold DESC;
"""
result = sqlite_client.execute_query(query)
result_df = pd.DataFrame(result.data)
result_df

INFO:src.clients.db_client:Query executed successfully. Returned 3 rows.


,payment_method,cars_sold
0,Financing,5
1,Cash,3
2,Lease,2


### Task 7: Popular Models by Region
- Which car models are most frequently sold in each region?

In [28]:
query = """
SELECT
    d.region,
    m.model_name,
    COUNT(*) AS sales_count
FROM sales s
JOIN dealerships d ON s.dealership_id = d.dealership_id
JOIN models m ON s.model_id = m.model_id
GROUP BY d.region, m.model_name
ORDER BY d.region, sales_count DESC;
"""
result = sqlite_client.execute_query(query)
result_df = pd.DataFrame(result.data)
result_df

INFO:src.clients.db_client:Query executed successfully. Returned 10 rows.


,region,model_name,sales_count
0,Asia Pacific,Taycan Cross Turismo,1
1,Asia Pacific,Macan,1
2,Asia Pacific,718 Boxster,1
3,Europe,Taycan,1
4,Europe,Cayenne Coupe,1
5,Europe,Cayenne,1
6,Europe,911 GT3,1
7,Middle East,Panamera,1
8,North America,911 Turbo S,1
9,North America,911 Carrera,1


### Task 8: Revenue by Dealership
- Calculate total sales revenue for each dealership. Sort by revenue descending

In [29]:
query = """
SELECT
    d.name AS dealership_name,
    SUM(s.price) AS total_revenue
FROM sales s
JOIN dealerships d ON s.dealership_id = d.dealership_id
GROUP BY d.dealership_id, d.name
ORDER BY total_revenue DESC;
"""
result = sqlite_client.execute_query(query)
result_df = pd.DataFrame(result.data)
result_df

INFO:src.clients.db_client:Query executed successfully. Returned 10 rows.


,dealership_name,total_revenue
0,Porsche Los Angeles,215000.0
1,Porsche Dubai,182000.0
2,Porsche Munich,167500.0
3,Porsche New York City,110500.0
4,Porsche Tokyo,98500.0
5,Porsche Berlin,92400.0
6,Porsche Paris,79900.0
7,Porsche London,72500.0
8,Porsche Melbourne,65000.0
9,Porsche Shanghai,59800.0


### Task 9: Average Age of Customers per Country
- What is the average customer age per country based on `date_of_birth`?

In [30]:
query = """
SELECT
    country,
    AVG((julianday('now') - julianday(date_of_birth)) / 365.25) AS average_age
FROM customers
WHERE date_of_birth IS NOT NULL
GROUP BY country
ORDER BY average_age DESC;
"""
result = sqlite_client.execute_query(query)
result_df = pd.DataFrame(result.data)
result_df

INFO:src.clients.db_client:Query executed successfully. Returned 8 rows.


,country,average_age
0,France,54.701055
1,United States,53.625079
2,Australia,52.773608
3,China,49.384149
4,Japan,45.890651
5,Germany,43.107626
6,United Kingdom,41.321178
7,United Arab Emirates,37.006325


### Task 10: Service Cost by Dealership
- Calculate the total service cost provided by each dealership

In [32]:
query = """
SELECT
    dealership_id,
    SUM(cost) AS total_service_cost
FROM service_records
GROUP BY dealership_id
ORDER BY total_service_cost DESC;
"""
result = sqlite_client.execute_query(query)
result_df = pd.DataFrame(result.data)
result_df

INFO:src.clients.db_client:Query executed successfully. Returned 10 rows.


,dealership_id,total_service_cost
0,7,3800.0
1,4,2800.0
2,5,1450.0
3,10,1350.0
4,9,1200.0
5,2,1200.0
6,8,1100.0
7,6,950.0
8,3,950.0
9,1,850.0


### Task 11: Model Price vs Horsepower
- Get average base price and average horsepower for each car `body_type`

In [ ]:
query = """
SELECT
    body_type,
    AVG(base_price) AS avg_base_price,
    AVG(horsepower) AS avg_horsepower
FROM models
GROUP BY body_type
ORDER BY body_type;
"""
result = sqlite_client.execute_query(query)
result_df = pd.DataFrame(result.data)
result_df

INFO:src.clients.db_client:Query executed successfully. Returned 6 rows.


,body_type,avg_base_price,avg_horsepower
0,Convertible,451666.666667,597.333333
1,Coupe,132450.000000,455.250000
2,SUV,61950.000000,291.500000
3,SUV Coupe,76500.000000,335.000000
4,Sedan,87550.000000,423.500000
5,Wagon,93700.000000,469.000000


### Task 12: Top Performing Dealerships
- Which dealerships have the highest average rating and highest sales revenue combined?

In [33]:
query = """
SELECT
    d.dealership_id,
    d.name,
    d.region,
    d.rating AS average_rating,
    COALESCE(SUM(s.price), 0) AS total_sales_revenue,
    d.rating + COALESCE(SUM(s.price), 0) AS combined_score
FROM dealerships d
LEFT JOIN sales s ON d.dealership_id = s.dealership_id
GROUP BY d.dealership_id, d.name, d.region, d.rating
ORDER BY combined_score DESC
LIMIT 10;  -- optional, top 10
"""
result = sqlite_client.execute_query(query)
result_df = pd.DataFrame(result.data)
result_df

INFO:src.clients.db_client:Query executed successfully. Returned 10 rows.


,dealership_id,name,region,average_rating,total_sales_revenue,combined_score
0,3,Porsche Los Angeles,North America,4.6,215000.0,215004.6
1,6,Porsche Dubai,Middle East,4.8,182000.0,182004.8
2,8,Porsche Munich,Europe,4.7,167500.0,167504.7
3,1,Porsche New York City,North America,4.7,110500.0,110504.7
4,4,Porsche Tokyo,Asia Pacific,4.9,98500.0,98504.9
5,2,Porsche Berlin,Europe,4.8,92400.0,92404.8
6,10,Porsche Paris,Europe,4.5,79900.0,79904.5
7,5,Porsche London,Europe,4.5,72500.0,72504.5
8,7,Porsche Melbourne,Asia Pacific,4.6,65000.0,65004.6
9,9,Porsche Shanghai,Asia Pacific,4.4,59800.0,59804.4


### Task 13: Profit Margin by Model
- Calculate average profit margin per car model by subtracting `base_price` from `average sale price`

In [ ]:
query = """
SELECT
    m.model_id,
    m.model_name,
    AVG(s.price) - m.base_price AS average_profit_margin
FROM models m
JOIN sales s ON m.model_id = s.model_id
GROUP BY m.model_id, m.model_name, m.base_price
ORDER BY average_profit_margin DESC;
"""
result = sqlite_client.execute_query(query)
result_df = pd.DataFrame(result.data)
result_df

INFO:src.clients.db_client:Query executed successfully. Returned 10 rows.


,model_id,model_name,average_profit_margin
0,4,Panamera,93600.0
1,1,911 Carrera,9300.0
2,2,911 Turbo S,8000.0
3,11,911 GT3,6400.0
4,3,Taycan,5700.0
5,6,Macan,4900.0
6,9,Taycan Cross Turismo,4800.0
7,5,Cayenne,3500.0
8,10,Cayenne Coupe,3400.0
9,7,718 Boxster,3000.0


### Task 14: Repeat Customers Analysis
- How many customers have purchased more than one car? List their customer IDs and number of purchases

In [ ]:
query = """
SELECT
    customer_id,
    COUNT(*) AS purchase_count
FROM sales
GROUP BY customer_id
HAVING COUNT(*) > 1;
"""
result = sqlite_client.execute_query(query)
result_df = pd.DataFrame(result.data)
result_df

INFO:src.clients.db_client:Query executed successfully. Returned 0 rows.


""


### Task 15: Service Frequency by Model
- For each car model, how many service records exist per 10,000 km on average?

In [ ]:
query = """
SELECT
    s.model_id,
    COUNT(sr.service_id) AS total_service_records,
    SUM(sr.mileage) AS total_mileage,
    (COUNT(sr.service_id) * 10000.0) / NULLIF(SUM(sr.mileage), 0) AS service_records_per_10000_km
FROM service_records sr
JOIN sales s ON sr.vin = s.vin
GROUP BY s.model_id;
"""
result = sqlite_client.execute_query(query)
result_df = pd.DataFrame(result.data)
result_df

INFO:src.clients.db_client:Query executed successfully. Returned 10 rows.


,model_id,total_service_records,total_mileage,service_records_per_10000_km
0,1,1,5000,2.000000
1,2,1,7500,1.333333
2,3,1,8000,1.250000
3,4,1,5000,2.000000
4,5,1,15000,0.666667
5,6,1,6000,1.666667
6,7,1,12000,0.833333
7,9,1,10000,1.000000
8,10,1,10000,1.000000
9,11,1,9000,1.111111


### Task 16: Lifetime Value by Customer
- Calculate the lifetime value of each customer based on total purchases and service costs. Sort top 10

In [ ]:
query = """
SELECT
    c.customer_id,
    COALESCE(SUM(s.price), 0) AS total_purchase_amount,
    COALESCE(SUM(sr.cost), 0) AS total_service_cost,
    COALESCE(SUM(s.price), 0) + COALESCE(SUM(sr.cost), 0) AS lifetime_value
FROM customers c
LEFT JOIN sales s ON c.customer_id = s.customer_id
LEFT JOIN service_records sr ON sr.vin IN (
    SELECT vin FROM sales WHERE customer_id = c.customer_id
)
GROUP BY c.customer_id
ORDER BY lifetime_value DESC
LIMIT 10;
"""
result = sqlite_client.execute_query(query)
result_df = pd.DataFrame(result.data)
result_df

INFO:src.clients.db_client:Query executed successfully. Returned 10 rows.


,customer_id,total_purchase_amount,total_service_cost,lifetime_value
0,3,215000.0,950.0,215950.0
1,6,182000.0,950.0,182950.0
2,8,167500.0,1100.0,168600.0
3,1,110500.0,850.0,111350.0
4,4,98500.0,2800.0,101300.0
5,2,92400.0,1200.0,93600.0
6,10,79900.0,1350.0,81250.0
7,5,72500.0,1450.0,73950.0
8,7,65000.0,3800.0,68800.0
9,9,59800.0,1200.0,61000.0
